In [3]:
import os
from pathlib import Path
from PIL import Image
# import geopandas as gpd
import itertools
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from branca.element import Template, MacroElement

from my_functions_idealisedprofiles import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Define whether to filter out values <0.1
remove_little_values = True

model_directory = '../../../../FloodModelling/MeganModel_New/'

### Read in landcover data

In [4]:
landcover_binary_fp =  model_directory +"LandCover_clipped_reclassified.tif"
landcover_full_fp =  model_directory +"LandCover_clipped.tif"

### Read in flood data

In [5]:
fp_feh= '../../../../FloodModelling/MeganModel_New/6h_feh_sp/{} (Max).Resampled.Terrain.tif'
fp_sp_10 = '../../../../FloodModelling/MeganModel_New/6h_sp_+10/{} (Max).Resampled.Terrain.tif'
fp_b2 = '../../../../FloodModelling/MeganModel_New/6h_sp_bl_0.9/{} (Max).Resampled.Terrain.tif'
fp_f2= '../../../../FloodModelling/MeganModel_New/6h_sp_fl_0.1/{} (Max).Resampled.Terrain.tif'
fp_f1= '../../../../FloodModelling/MeganModel_New/6h_sp_fl_0.3/{} (Max).Resampled.Terrain.tif'

depth_b2, out_meta = prepare_rainfall_scenario_raster(fp_b2.format('Depth'), remove_little_values)
depth_feh = prepare_rainfall_scenario_raster(fp_feh.format('Depth'), remove_little_values)[0]
depth_sp_10 = prepare_rainfall_scenario_raster(fp_sp_10.format('Depth'), remove_little_values)[0]
depth_f2 = prepare_rainfall_scenario_raster(fp_f2.format('Depth'), remove_little_values)[0]
depth_f1 = prepare_rainfall_scenario_raster(fp_f1.format('Depth'), remove_little_values)[0]

### Set all values over 0.1 to 1, and rest to 0

In [6]:
depth_feh = np.where(depth_feh>=0.1, 1, 0)
depth_sp_10= np.where(depth_sp_10>=0.1, 1, 0)
depth_b2 = np.where(depth_b2 >=0.1, 1, 0)
depth_f2 = np.where(depth_f2>=0.1, 1, 0)
depth_f1 = np.where(depth_f1 >=0.1, 1, 0)

### Find difference between the FEH and another profile (ones which are off the 1:1 line on scatter plot).

In [6]:
diff_feh_b2 = depth_b2 - depth_feh 
diff_f1_b2 = depth_b2 - depth_f1 

### Set 0s to np.nan ()

In [7]:
depth_feh = np.where(depth_feh==0, np.nan, depth_feh)
depth_sp_10 = np.where(depth_sp_10==0, np.nan, depth_sp_10)
depth_b2 = np.where(depth_b2==0, np.nan, depth_b2)
depth_f1 = np.where(depth_f1==0, np.nan, depth_f1)

### Test whether processing has worked as expected

In [8]:
def colorize(array, cmap):
    cm = plt.cm.get_cmap(cmap)    
    if np.nanmin(array) == np.nanmax(array):
        return np.uint8(cm(array)  * 255)
    else:
        normed_data = (array - np.nanmin(array)) / (np.nanmax(array) - np.nanmin(array)) 
        return np.uint8(cm(normed_data)  * 255)


In [9]:
# np.nanmax(diff[np.isfinite(diff)])
# np.histogram(depth_f1[np.isfinite(depth_f1)])
np.unique(depth_f1)

array([ 1., nan])

### Where difference is 0, set to np.nan

In [10]:
# Difference in scatter plot
diff_feh_b2 = np.where(diff_feh_b2==0, np.nan, diff_feh_b2)
# No difference in scatter plot
diff_f1_b2 = np.where(diff_f1_b2==0, np.nan, diff_f1_b2)

### Define colourmaps

In [8]:
cmap_landcover_binary = mpl.colors.ListedColormap(['grey', 'white'])
cmap_diff = mpl.colors.ListedColormap(['purple', 'darkorange', 'green'])
cmap = mpl.colors.ListedColormap(['white', 'blue'])
cmap_landcover_full = mpl.colors.ListedColormap(['darkslategrey', 'khaki', 'darkgreen','darkturquoise','#59F74D', 'slateblue', 'lightgreen', 'palegreen', 'pink'])

### Save rasters (needed for plotting)

In [10]:
# save_array_as_raster(diff_feh_b2, '../../../../FloodModelling/MeganModel_New/diff_feh_b2.tif', out_meta) 
# save_array_as_raster(diff_f1_b2, '../../../../FloodModelling/MeganModel_New/diff_f1_b2.tif', out_meta) 

save_array_as_raster(depth_sp_10, '../../../../FloodModelling/MeganModel_New/depth_sp_10_test.tif', out_meta) 
save_array_as_raster(depth_feh, '../../../../FloodModelling/MeganModel_New/depth_feh_test.tif', out_meta) 
save_array_as_raster(depth_b2, '../../../../FloodModelling/MeganModel_New/depth_b2_test.tif', out_meta) 
save_array_as_raster(depth_f1, '../../../../FloodModelling/MeganModel_New/depth_f1_test.tif', out_meta) 

### Set up for plotting

In [20]:
# dict_of_fps_and_names = {'landcover_full': landcover_full_fp,
#                         'landcover_binary': landcover_binary_fp,
#                         "diff_f1_b2": '../../../../FloodModelling/MeganModel_New/diff_f1_b2.tif',
#                          "diff_feh_b2": '../../../../FloodModelling/MeganModel_New/diff_feh_b2.tif',
#                          "b2": '../../../../FloodModelling/MeganModel_New/depth_b2_test.tif',
#                          "f1":'../../../../FloodModelling/MeganModel_New/depth_f1_test.tif',
#                         "feh":'../../../../FloodModelling/MeganModel_New/depth_feh_test.tif'}    


dict_of_fps_and_names = {'landcover_binary': landcover_binary_fp,
                         "sp10":"../../../../FloodModelling/MeganModel_New/depth_sp_10_test.tif",
                        "feh":"../../../../FloodModelling/MeganModel_New/depth_feh_test.tif"}    


dict_of_cmaps ={'landcover_binary': cmap_landcover_binary, "sp10":cmap, 'landcover_full': cmap_landcover_full, "diff_feh_b2":cmap_diff,
                "diff_f1_b2":cmap_diff,"b2":cmap, "feh":cmap , "f1":cmap }

### Plot with folium

In [21]:
# Set up figure
f = folium.Figure(width=800, height=700)

# Create base map - location figures were from clat, clon, but wanted to create map before loop
mapa = folium.Map(location=[53.768306874761016, -1.3756056884868098],zoom_start=13).add_to(f)
folium.TileLayer(
    opacity=1,
    tiles = 'https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}',
    attr="No Base Map",
    name="No Base Map",
    show=True
).add_to(mapa)

# Add to map
catchment_boundary_feature_group = FeatureGroup(name='Catchment boundary')
catchment_boundary_feature_group.add_child(folium.GeoJson(data=catchment_gdf["geometry"], style_function=lambda x, 
                                                          fillColor='#00000000', color='Black': {
        "fillColor": '#00000000',"color": 'Black',}))

# Add raster data
for name,fp in dict_of_fps_and_names.items():
    
    # read in with xarray and convert projection
    xarray_dataarray = prep_for_folium_plotting(fp)
    # Get coordinates needed in plotting
    clat, clon = xarray_dataarray.y.values.mean(), xarray_dataarray.x.values.mean()
    mlat, mlon = xarray_dataarray.y.values.min(), xarray_dataarray.x.values.min()
    xlat, xlon = xarray_dataarray.y.values.max(), xarray_dataarray.x.values.max()
    # Apply colormap
    data  = ma.masked_invalid(xarray_dataarray.values)
    colored_data = colorize(data.data, cmap=dict_of_cmaps[name])
    # Add to map
    feature_group1 = FeatureGroup(name=name)
    feature_group1.add_child(folium.raster_layers.ImageOverlay(colored_data,
                              [[mlat, mlon], [xlat, xlon]],
                              opacity=1,interactive=True, popup=name))
    mapa.add_child(feature_group1)

# Add legend
macro = MacroElement()
macro._template = Template(template)
mapa.get_root().add_child(macro)

# Add layer control button
mapa.add_child(catchment_boundary_feature_group)
mapa.add_child(LayerControl("topright", collapsed = False))
display(mapa)    

In [11]:
template = """
{% macro html(this, kwargs) %}
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">
  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            left: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});
  </script>
</head>
<body>
 
<div id='maplegend' class='maplegend' 
    style='position: fixed; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; left: 20px; bottom: 20px;'>
<div class='legend-title'> Key </div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#800080.;opacity:1;'></span> B2 flooded (not FEH)</li>
    <li><span style='background:#008000;opacity:1;'></span> FEH flooded (not B2)</li>
  </ul>
</div>
</div>
 
</body>
</html>
<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""